# Periodic download of IEX stock-ticker data

## Read IEX API credentials from `~/.config/iex.ini`

In [1]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [2]:
# tickers = sorted("MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK HRB BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CBS CDW CE CELG CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HOG HIG HAS HCA HCP HP HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HPQ HUM HBAN HII IEX IDXX INFO ITW ILMN IR INTC ICE IBM INCY IP IPG IFF INTU ISRG IVZ IPGP IQV IRM JKHY JEC JBHT SJM JNJ JCI JPM JNPR KSU K KEY KEYS KMB KIM KMI KLAC KSS KHC KR LB LHX LH LRCX LW LVS LEG LDOS LEN LLY LNC LIN LKQ LMT L LOW LYB MTB MAC M MRO MPC MKTX MAR MMC MLM MAS MA MKC MXIM MCD MCK MDT MRK MET MTD MGM MCHP MU MSFT MAA MHK TAP MDLZ MNST MCO MS MOS MSI MSCI MYL NDAQ NOV NTAP NFLX NWL NEM NWSA NWS NEE NLSN NKE NI NBL JWN NSC NTRS NOC NCLH NRG NUE NVDA NVR ORLY OXY OMC OKE ORCL PCAR PKG PH PAYX PYPL PNR PBCT PEP PKI PRGO PFE PM PSX PNW PXD PNC PPG PPL PFG PG PGR PLD PRU PEG PSA PHM PVH QRVO PWR QCOM DGX RL RJF RTN O REG REGN RF RSG RMD RHI ROK ROL ROP ROST RCL CRM SBAC SLB STX SEE SRE SHW SPG SWKS SLG SNA SO LUV SPGI SWK SBUX STT SYK STI SIVB SYMC SYF SNPS SYY TMUS TROW TTWO TPR TGT TEL FTI TFX TXN TXT TMO TIF TWTR TJX TSCO TDG TRV TRIP TSN UDR ULTA USB UAA UA UNP UAL UNH UPS URI UTX UHS UNM VFC VLO VAR VTR VRSN VRSK VZ VRTX VIAB V VNO VMC WAB WMT WBA DIS WM WAT WEC WCG WFC WELL WDC WU WRK WY WHR WMB WLTW WYNN XEL XRX XLNX XYL YUM ZBH ZION ZTS".split(" "))
# aapl = tickers.index('AAPL'); aapl
ticker = 'AAPL'
tickers = [ticker]
num_tickers = len(tickers)
num_tickers

1

In [11]:
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()
today.strftime('%Y-%m-%d')

'2019-11-05'

In [4]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [5]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [67]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = date(2019, 4, 1)
end_date = today + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 157 days ([2019-04-01,2019-11-06)), 390 minutes per day


In [120]:
def fetch_minute(ticker):
    n = dt.now()
    date_str = n.strftime('%Y%m%d')
    time_str = n.strftime('%Y%m%dT%H%M%S')
    out_path = data_dir / ('%s-%s' % (time_str, ticker))
    if out_path.exists():
        print('Found %s' % out_path)

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    print('%s: found %d data' % (time_str, len(data)))
    with out_path.open('wb') as f:
        f.write(resp.content)

In [117]:
from time import sleep

In [129]:
while True:
    fetch_minute('AAPL')
    second = dt.now().second
    if second >= 23 and second < 27:
        sleep(1)
    else:
        s = (60-second) % 10
        if s == 0:
            s = 10
        if second < 23 and second + s > 23:
            sleep(23 - second)
        else:
            sleep(s)

20191105T144533: found 15 data
20191105T144540: found 15 data
20191105T144550: found 15 data
20191105T144600: found 15 data
20191105T144610: found 15 data
20191105T144620: found 15 data
20191105T144623: found 15 data
20191105T144625: found 16 data
20191105T144626: found 16 data
20191105T144627: found 16 data
20191105T144630: found 16 data
20191105T144640: found 16 data
20191105T144650: found 16 data
20191105T144700: found 16 data
20191105T144710: found 16 data
20191105T144720: found 16 data
20191105T144723: found 16 data
20191105T144724: found 16 data
20191105T144725: found 17 data
20191105T144726: found 17 data
20191105T144730: found 17 data


KeyboardInterrupt: 

In [32]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [25]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [34]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
    for ticker in tickers
); None

CPU times: user 594 ms, sys: 231 ms, total: 825 ms
Wall time: 8.83 s


In [35]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    sqrt, log
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [37]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [68]:
def load_ticker_date_df(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [69]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [138]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 4.36 s, sys: 147 ms, total: 4.5 s
Wall time: 4.7 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.600,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.98,190.980,190.640,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.93,191.090,190.780,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.83,191.010,190.760,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.70,190.760,190.600,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-11-04 15:55:00,257.540,257.38,257.540,257.350,257.413,1212,311984.980,20
2019-11-04 15:56:00,257.370,257.38,257.460,257.360,257.427,4916,1265513.105,43
2019-11-04 15:57:00,257.290,257.39,257.390,257.230,257.280,4515,1161617.860,40


Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [139]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [140]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [141]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
...,...,...,...,...,...,...,...,...,...
2019-11-04 13:08:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.310
2019-11-04 13:22:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.510
2019-11-04 13:46:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.900


Spot-check: consecutive pairs of minutes where no trades occurred

In [142]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-23 14:16:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,202.66
...,...,...,...,...,...,...,...,...,...
2019-10-28 13:28:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-10-28 13:55:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,248.51
2019-10-28 13:56:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [143]:
# aapl['date'] = datetime.apply(to_date)
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
# aapl.drop(columns='date', inplace=True)
aapl.count()

open              55961
close             55961
high              55961
low               55961
average           55961
volume            56757
notional          56757
numberOfTrades    56757
prev_close        56757
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [144]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [145]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
...,...,...,...,...,...,...,...,...,...
2019-11-04 13:46:00,256.900,256.900,256.900,256.900,256.900,0,0.000,0,256.900
2019-11-04 15:12:00,257.130,257.130,257.130,257.070,257.098,605,155544.450,7,257.135
2019-11-04 15:13:00,257.130,257.130,257.130,257.130,257.130,0,0.000,0,257.130


Copy the data, and make `open` a log-fold change over the previous close, and other intra-minute features a log-fold change over their corresponding opening price:

In [146]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
2019-11-04 13:47:00,0.000311,-0.000078,0.000039,-0.000078,0.000027
2019-11-04 15:13:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-11-04 15:14:00,-0.000194,-0.000370,0.000000,-0.000389,-0.000280


In [147]:
lg.count()

open              56757
close             56757
high              56757
low               56757
average           56757
volume            56757
notional          56757
numberOfTrades    56757
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [148]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-11-04 15:55:00,-0.000116,-0.000621,0.000000,-0.000738,-0.000493,4.472136,4.120662
2019-11-04 15:56:00,-0.000039,0.000039,0.000350,-0.000039,0.000221,6.557439,4.747759
2019-11-04 15:57:00,-0.000350,0.000389,0.000389,-0.000233,-0.000039,6.324555,4.735101


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [151]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,1.729424e-03
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-2.618418e-04
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-5.238894e-04
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-6.814668e-04
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,7.862666e-04
...,...,...,...,...,...,...,...,...
2019-11-04 15:53:00,0.000078,-0.000504,0.000116,-0.000582,-0.000016,4.582576,4.752769,-3.493585e-04
2019-11-04 15:54:00,-0.000058,-0.000291,0.000019,-0.000446,-0.000260,5.000000,4.352598,-7.379358e-04
2019-11-04 15:55:00,-0.000116,-0.000621,0.000000,-0.000738,-0.000493,4.472136,4.120662,1.461640e-17


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [153]:
lg.dropna(how='any', inplace=True)
lg.count()

open          56465
close         56465
hi            56465
lo            56465
avg           56465
trades        56465
avgVol        56465
next_close    56465
dtype: int64

In [ ]:
from random import random, seed
validation_split = 0.1

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_flag(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random() < validation_split

vfs = datetime.apply(get_validation_flag).rename('validation'); vfs
nt, nv = vfs[~vfs].count(), vfs[vfs].count()
n = nt + nv
nt / n, nv / n

In [ ]:
lg_t = lg[~vfs]
lg_v = lg[ vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

In [ ]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [ ]:
desc(lg)

In [ ]:
normed = lg.copy()
for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

In [ ]:
desc(normed)

In [ ]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

In [ ]:
cols = normed.columns

In [ ]:
col = 'open'
arr = concat([ normed[col].shift(i) for i in reversed(range(window)) ], axis=1).fillna(means[col]).values
arr.shape

In [ ]:
arr[:10]

In [ ]:
window = 30
arr = np.moveaxis(
    array([ 
        concat([ normed[col].shift(i) for i in reversed(range(window)) ], axis=1).fillna(means[col]).values
        for col in cols 
    ]),
    0, 2
)
arr.shape    

In [ ]:
n = arr.shape[0]
perm = permutation(n)
perm

In [ ]:
normed

In [ ]:
vfs = normed.merge(vfs, how='left', left_index=True, right_index=True).validation; vfs

In [ ]:
vx = arr[vfs]
vy = y[vfs]
tx = arr[~vfs]
ty = y[~vfs]
ax = np.concatenate([tx, vx])
ay = np.concatenate([ty, vy])
[ a.shape for a in [ tx,ty, vx,vy, ax,ay ] ]

In [ ]:
tn = len(tx)
vn = len(vx)
tn, vn

In [ ]:
tf = tn / n
vf = vn / n
tf, vf

In [ ]:
divd = concat([ (aapl[col] / aapl.prev_close).rename(col) for col in fill_cols ], axis=1); divd

In [ ]:
desc(divd)

In [ ]:
aapl[divd.close > 1.015]

In [ ]:
x = ax
oy = y
y = ay

-------

In [ ]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras import Sequential

In [ ]:
len(cols)

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(window, len(cols))))
model.add(Dense(1))
model.build()
model.compile(loss='mae', optimizer='adam')
model.summary()

In [ ]:
px = model.predict(vx)
px.max() - px.min(), x[:10, -1, 0], y[:10], px[:10, -1]

In [ ]:
mean(abs(tx[:, -1, avg] - ty)), mean(abs(vx[:, -1, avg] - vy)), mean(abs(x[:, -1, avg] - y))

In [ ]:
vn, tn, n

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=10000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=10000,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=n,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

In [ ]:
def diffs_df():
    tpx = model.predict(tx)[:, 0]
    tds = ty-tpx
    tds1 = abs(tds)
    tds2 = tds**2

    vpx = model.predict(vx)[:, 0]
    vds = vy-vpx
    vds1 = abs(vds)
    vds2 = vds**2

    px = np.concatenate([tpx, vpx])
    ds = y-px
    ds1 = abs(ds)
    ds2 = ds**2

    df = desc_df(
        DF([ 
            describe(a) 
            for a in [ 
                y, ty, vy, 
                px, ds, ds1, ds2, 
                tpx, tds, tds1, tds2, 
                vpx, vds, vds1, vds2, 
            ] 
        ])
    )
    df['name'] = [ 
        'y', 'ty', 'vy', 
        'px', 'ds', 'ds1', 'ds2', 
        'tpx', 'tds', 'tds1', 'tds2', 
        'vpx', 'vds', 'vds1', 'vds2', 
    ]
    df.set_index('name', inplace=True)
    return df

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=10)
diffs_df()

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)
diffs_df()

In [ ]:
%%time
model.fit(x, y, 
          validation_split=vf,
          batch_size=100000,
          epochs=1000)
diffs_df()

In [ ]:
DF(model.history.history)

In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)